In [ ]:
%pip install html2text
%pip install fasttext
%pip install beautifulsoup4

In [ ]:
import requests
import json
from tqdm import tqdm
from functools import reduce

from bs4 import BeautifulSoup
import numpy as np
from datasets import load_dataset
from langchain_milvus.utils.sparse import BM25SparseEmbedding

from create_habr import parse_post, parse_comments

dataset = load_dataset('IlyaGusev/habr', 
                       split="train", 
                       streaming=True,
                       trust_remote_code=True)


samples = []

index = 0
for sample in dataset:
    samples.append(sample['text_markdown'])
    index += 1

    if index == 1000:
        break


bm25 = BM25SparseEmbedding(corpus=samples, language="ru")

c:\Users\kshch\Projects\Habr_RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
query = "Ельцин захватил власть в СССР и стал первым президентом России"
term_importance = {}

for term in bm25.analyzer.tokenizer.tokenize(query):
    importances = bm25.bm25_ef.encode_queries([term]).toarray()[0]
    indexes = np.where(importances != 0)[0]
    index = indexes[0] if len(indexes) != 0 else 0
    importance = importances[index]

    term_importance[term] = importance.item()

term_importance = {k: v for k, v in sorted(term_importance.items(), key=lambda item: item[1], reverse=True)}
key_words = list(term_importance.keys())[:3]

In [ ]:
query = reduce(lambda a, b: a + "+" + b, key_words)
url = f"https://habr.com/ru/search/?q={query}&target_type=posts&order=rating"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

ids = []
for index, article in enumerate(soup.find_all('article')):
    if index < 5:
        ids.append(article.get('id'))

In [ ]:
records = []

for post_id in tqdm(ids):
    record = parse_post(post_id)
    if not record:
        continue
    comments = parse_comments(post_id)
    record["comments"] = comments
    records.append(record)

json_object = json.dumps(records) 
with open("output.json", "w") as outfile:
    outfile.write(json_object)


with open('output.json', 'r') as openfile:
    docs = json.load(openfile)
 
texts = []
for doc in docs:
    text = parse_doc(doc)
    texts.append(text)

In [ ]:
import os
import logging
from dotenv import load_dotenv
from langchain_milvus.retrievers import MilvusCollectionHybridSearchRetriever
from pymilvus import (
    MilvusClient,
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    WeightedRanker,
    connections,
)


load_dotenv()

CONNECTION_URI = 'http://localhost:19530'
connections.connect(uri=CONNECTION_URI)

dense_embedding_func = embedder
sparse_embedding_func = bm25

pk_field = "doc_id"
dense_field = "dense_vector"
sparse_field = "sparse_vector"
text_field = "text"

fields = [
    FieldSchema(
        name=pk_field,
        dtype=DataType.VARCHAR,
        is_primary=True,
        auto_id=True,
        max_length=100,
    ),
    FieldSchema(name=dense_field, dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name=sparse_field, dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name=text_field, dtype=DataType.VARCHAR, max_length=65_535),
]


collaction_name = "habr_collection"
schema = CollectionSchema(fields=fields, enable_dynamic_field=False)
collection = Collection(
    name=collaction_name, schema=schema, consistency_level="Strong"
)


dense_index = {"index_type": "DISKANN", "metric_type": "IP"}
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}

collection.create_index("dense_vector", dense_index)
collection.create_index("sparse_vector", sparse_index)

collection.flush()


entities = []
for index, doc in enumerate(corpus):
    print('Iteration:', index+1)
    text = doc
    entity = {
        dense_field: embedder.embed_query(text),
        sparse_field: bm25.embed_query(text),
        text_field: text,
    }
    entities.append(entity)

collection.insert(entities)
collection.load()

sparse_search_params = {"metric_type": "IP"}
dense_search_params = {"metric_type": "IP", "params": {}}

retriever = MilvusCollectionHybridSearchRetriever(
    collection=collection,
    rerank=WeightedRanker(0.8, 0.2),
    anns_fields=[dense_field, sparse_field],
    field_embeddings=[dense_embedding_func, sparse_embedding_func],
    field_search_params=[dense_search_params, sparse_search_params],
    top_k=5,
    text_field=text_field,
)

In [139]:
retriever.invoke(sample_1)

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.06it/s]


[Document(metadata={'doc_id': '454520780631575101'}, page_content='Всем привет!\nМы закончили обрабатывать видео с конференции C++ Siberia 2015 и собрали их в плейлисте на youtube. Также они доступны на странице конференции.\nА ещё у нас есть замечательные новости про конференцию C++ Russia 2016 в Санкт-Петербурге. Вкратце: все будет очень круто.\nИтак, конференция пройдёт 26-27 февраля в Санкт-Петербурге (Park Inn by Radisson Прибалтийская). Мы пока собираем программу и все ещё принимаем заявки на доклады. Если у Вас есть о чём рассказать, пишите. Если есть идея, но Вы не уверены, пишите, обсудим. Также мы не оставляем идеи сделать блиц-доклады.Если у Вас есть тема для обсуждения, но Вы не считаете достаточно серьёзной для целого доклада, давайте обсудим её тесной компанией таких же любопытных!\nКак всегда на мы собираем докладчиков со всего мира. В этот раз в числе прочих у нас выступят:\n   Hartmut Kaiser — контрибьютор boost, один из авторов HPX\n   Kirk Shoop — автор библиотеки Rx

In [140]:
sample_1

'Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.\nЧастично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.\nФормат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.\nvia AP'

In [141]:
text = sample_1
entity = {
    dense_field: embedder.embed_query(text),
    sparse_field: bm25.embed_query(text),
    text_field: text,
}

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


In [142]:
collection.insert(entity)

(insert count: 1, delete count: 0, upsert count: 0, timestamp: 454520907439800323, success count: 1, err count: 0

In [143]:
collection.load()

In [144]:
retriever.invoke(sample_1)

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.20it/s]


[Document(metadata={'doc_id': '454520780631575151'}, page_content='Абитуриенты, поступающие в Университет Чикаго, уже начиная с этой осени, будут обязаны, помимо сдачи основных экзаменов, представить четырехстраничные презентации в формате PowerPoint — на свободную тему.\nЧастично новые требования связаны с тем, что PowerPoint становится одним из бизнес-инструментов, а кроме того, считают авторы инициативы, в презентации абитуриент сможет раскрыть свои новые качества, которые не смогут проявиться на традиционных экзаменах.\nФормат PowerPoint стал своего рода языком делового общения. Ежедневно, по оценкам Microsoft, в мире показывается порядка 30 млн презентаций.\nvia AP'),
 Document(metadata={'doc_id': '454520780631575101'}, page_content='Всем привет!\nМы закончили обрабатывать видео с конференции C++ Siberia 2015 и собрали их в плейлисте на youtube. Также они доступны на странице конференции.\nА ещё у нас есть замечательные новости про конференцию C++ Russia 2016 в Санкт-Петербурге. В